In [5]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import boto3
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session c7285399-dcf0-4d45-bf4b-693b5d7895ac.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session c7285399-dcf0-4d45-bf4b-693b5d7895ac.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session c7285399-dcf0-4d45-bf4b-693b5d7895ac.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: None
Setting new worker type to: G.1X


You are already connected to a glueetl session c7285399-dcf0-4d45-bf4b-693b5d7895ac.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: None
Setting new number of workers to: 5



In [2]:
from pyspark.sql.functions import *
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

In [3]:
s3_path = "s3://spotify-bucket-etl-ashutosh/raw-data/to-be-processed-data/"
source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type = "s3",
    connection_options = {"paths":[s3_path]},
    format = "json"
)

In [4]:
source_df = source_dyf.toDF()
df = source_df
df.withColumn("items",explode("items")).show(5)

+--------------------+--------------------+-----+----+------+--------+-----+
|                href|               items|limit|next|offset|previous|total|
+--------------------+--------------------+-----+----+------+--------+-----+
|https://api.spoti...|{2024-08-22T09:30...|  100|null|     0|    null|   50|
|https://api.spoti...|{2024-08-22T09:30...|  100|null|     0|    null|   50|
|https://api.spoti...|{2024-08-22T09:30...|  100|null|     0|    null|   50|
|https://api.spoti...|{2024-08-22T09:30...|  100|null|     0|    null|   50|
|https://api.spoti...|{2024-08-22T09:30...|  100|null|     0|    null|   50|
+--------------------+--------------------+-----+----+------+--------+-----+
only showing top 5 rows

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [5]:
def df_album(df):
    return df.withColumn("items",explode("items")).select(
    (col("items.track.album.id").alias("album_id")),
    (col("items.track.album.name").alias("album_name")),
    (col("items.track.album.external_urls.spotify").alias("external_urls")),
    (col("items.track.album.release_date").alias("release_dates")),
    (col("items.track.album.total_tracks").alias("total_tracks"))).drop_duplicates(['album_id'])
def df_artist(df):
    df_exploded = df.withColumn("items",explode("items"))
    df_artist_exploded = df_exploded.withColumn("items", explode("items.track.artists"))
    return df_artist_exploded.select(
    col("items.id").alias("artist_id"),
    col("items.name").alias("artist_name"),
    col("items.external_urls.spotify").alias("external_urls")).drop_duplicates(['artist_id'])
def df_songs(df):
    return df.withColumn("items",explode("items")).select(
    col("items.track.id").alias("song_id"),
    col("items.track.name").alias("song_name"),
    round(col("items.track.duration_ms")/60000,2).alias("song_duration"),
    col("items.track.popularity").alias("popularity")).drop_duplicates(['song_id'])

In [6]:
album_df = df_album(df)
album_df.show(5)

+--------------------+--------------------+--------------------+-------------+------------+
|            album_id|          album_name|       external_urls|release_dates|total_tracks|
+--------------------+--------------------+--------------------+-------------+------------+
|013jUXOfDFXnDMBet...|          Lost;Found|https://open.spot...|   2024-05-25|          14|
|0LcyzKKw3RjFKL6yg...|         Ek Tha Raja|https://open.spot...|   2024-03-18|          16|
|0a183xiCHiC1GQd8o...|              ANIMAL|https://open.spot...|   2023-11-24|           8|
|0cJXodCZCl2EWRNcw...|Tainu Khabar Nahi...|https://open.spot...|   2024-05-31|           1|
|0kZKLq2WZQWvXvbxv...|Janiye (from the ...|https://open.spot...|   2023-03-17|           1|
+--------------------+--------------------+--------------------+-------------+------------+
only showing top 5 rows


In [7]:
artist_df = df_artist(df)
artist_df.show(5)

+--------------------+---------------+--------------------+
|           artist_id|    artist_name|       external_urls|
+--------------------+---------------+--------------------+
|00sCATpEvwH48ays7...|  Jonita Gandhi|https://open.spot...|
|05dG9pMLRWenxzvSm...|       MixSingh|https://open.spot...|
|05etL4pzWd6TSv1x5...|Faheem Abdullah|https://open.spot...|
|09UmIX92EUH9hAK4b...|        Mithoon|https://open.spot...|
|0NErdIJtuKBjtxKml...|    Divya Kumar|https://open.spot...|
+--------------------+---------------+--------------------+
only showing top 5 rows


In [8]:
songs_df = df_songs(df)
songs_df.show(5)

+--------------------+--------------------+-------------+----------+
|             song_id|           song_name|song_duration|popularity|
+--------------------+--------------------+-------------+----------+
|0645eBDehHcqfiF15...|Janiye (from the ...|         3.72|        77|
|0M0ANKNzmM4Odd7FN...|Akhiyaan Gulaab (...|         2.85|        77|
|0MTdYgTZ25sLCO6kV...|Aasa Kooda - From...|         3.59|        81|
|0OA00aPt3BV10qeMI...|           Big Dawgs|         3.18|        91|
|0TL0LFcwIBF5eX7ar...|                Husn|         3.63|        78|
+--------------------+--------------------+-------------+----------+
only showing top 5 rows


In [9]:
def write_to_s3(df, path_suffix, format_type = "csv"):
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    glueContext.write_dynamic_frame.from_options(
    frame = dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": f"s3://spotify-bucket-etl-ashutosh/transformed-data/{path_suffix}/"},
    format = format_type
    )

In [10]:
write_to_s3(album_df, "album_data/transformed_album_{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")),"csv")
write_to_s3(artist_df, "artist_data/transformed_artist_{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")),"csv")
write_to_s3(songs_df, "songs_data/transformed_songs_{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")),"csv")


In [1]:
def list_s3_objects(buckets,prefix):
    s3_client = boto3.client('s3')
    response = s3_client.list_objects_v2(Bucket=buckets, Prefix=prefix)
    keys = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.json')]
    return keys

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: c7285399-dcf0-4d45-bf4b-693b5d7895ac
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session c7285399-dcf0-4d45-bf4b-693b5d7895ac to get into ready status...
Session c7285399-dcf0-4d45-bf4b-693b5d7895ac has been created.



In [21]:
bucket_name = "spotify-bucket-etl-ashutosh"
prefix = "raw-data/to-be-processed-data/"
spotify_keys = list_s3_objects(bucket_name,prefix)

In [22]:
def move_and_delete_files(spotify_keys,bucket):
    s3_resource = boto3.resource('s3')
    for key in spotify_keys:
        copy_source = {
            'Bucket': bucket,
            'Key': key
        }
        new_key = "raw-data/processed-data/" + key.split("/")[-1]
        s3_resource.meta.client.copy(copy_source, bucket, new_key)
        s3_resource.Object(bucket, key).delete()

In [23]:
move_and_delete_files(spotify_keys,bucket_name)

In [18]:
job.commit()